# Implementación del TSP en DEAP
por _Eugenio Scalise_, basado en el ejemplo [tsp.py](https://github.com/DEAP/deap/blob/master/examples/ga/tsp.py), disponible en el [repositorio de la biblioteca DEAP](https://github.com/DEAP/deap)

Adaptado para el curso electivo **Inteligencia Artificial**, Postgrado en Ciencias de la Computación, Facultad de Ciencias, Universidad Central de Venezuela. Julio 2019.

Para más detalles, revise la [documentación de DEAP](https://deap.readthedocs.io).

### Celda para instalar paquetes 

Los paquetes no disponibles en el ambiente donde se ejecuta el Jupyter (colab, por ejemplo), pueden ser instalados mediante la sentencia:

`pip install <nombre_del_paquete>`

Basta ejecutar la primera vez y luego comentar la celda.

In [1]:
#pip install deap

### Imports 

Si ocurre un error al ejecutar alguna de las celdas, descomente la celda anterior e instale el paquete correspondiente.

In [2]:
import array
import random
import json

import numpy

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

En la solución original, los datos del grafo TSP eran leídos desde un JSON desde el filesystem. Por simplicidad, para que este notebook se pueda ejecutar en cualquier ambiente (colab, por ejemplo), mofiqué el código original y leo el JSON desde un URL en el repositorio (URL del _raw file_).

Estas son las líneas eliminadas del código original:

```python
# gr*.json contains the distance map in list of list style in JSON format
# Optimal solutions are : gr17 = 2085, gr24 = 1272, gr120 = 6942
with open("tsp/gr17.json", "r") as tsp_data:
  tsp = json.load(tsp_data)
```

In [3]:
# Hack para leer el archivo desde un URL y no desde el file system
# gr*.json contiene el map de distancias como listas de listas en JSON
# Soluciones óptimas: gr17 = 2085, gr24 = 1272, gr120 = 6942

# Importando el objeto request para manipular el URL
import urllib.request

# Cambiar el URL según el caso de prueba que se quiera probar
# (el número denota el tamaño del circuito)
JSON_URL = "https://raw.githubusercontent.com/escalise/AI_Course_PG/master/notebooks/tsp/gr17.json"
tsp_data = urllib.request.urlopen(JSON_URL).read()
tsp = json.loads(tsp_data)

#print(tsp)

### Matriz de distancias y tamaño del tour/camino

In [4]:
distance_map = tsp["DistanceMatrix"]
IND_SIZE = tsp["TourSize"]
best_fitness = tsp["OptDistance"] # el JSON contiene el óptimo, útil para comparar 

### Inicializaciones/configuraciones de DEAP

(ver más detalles en el [tutorial de DEAP](https://deap.readthedocs.io/en/master/index.html) o el framework que decidan usar).

In [5]:
# el problema es de minimización, por eso se inicializa el atributo weights en -1.0
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

# los individuos se representan como arreglos y se especifica 
# que se va a minimizar 
# la función de aptitud/fitness
creator.create("Individual", array.array, typecode='i', fitness=creator.FitnessMin)

# el toolbox se usa para almacenar los operadores evolutivos
toolbox = base.Toolbox()

# registar el generador de atributos de los individuos
toolbox.register("indices", random.sample, range(IND_SIZE), IND_SIZE)
# inicializadores de las estructuras de individuos
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

### Definición de la función de adaptación/fitness

Se define en Python la función de aptitud que luego es registrada en el framework.

In [6]:
def evalTSP(individual):
    distance = distance_map[individual[-1]][individual[0]]
    for gene1, gene2 in zip(individual[0:-1], individual[1:]):
        distance += distance_map[gene1][gene2]
    return distance,

### Configuración del tipo de cruce, mutación, selección y función de adaptación


In [7]:
# leer la documentación
toolbox.register("mate", tools.cxPartialyMatched)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)

### Función `evolve()` con la dinámica evolutiva

El paquete `algorithms` contiene diversas implementaciones de algoritmos evolutivos. El método `eaSimple` ya contiene la dinámica evolutiva clásica vista en clases (selección, cruce y mutación), implementada según la configuración hecha. Se puede personalizar más colocando una dinámica propia (ver el tutorial y los ejemplos en el repositorio de DEAP en GitHub).

In [8]:
def evolve():
    random.seed(42) # al fijar las semillas todas las ejecuciones serán iguales

    # población
    pop = toolbox.population(n=300)

    # mejor individuo (opcional, se puede tomar de las estadísticas)
    hof = tools.HallOfFame(1)
    
    # DEAP provee estadísticas a calcular en la corrida. 
    # En este ejemplo se calcula el promedio y desviación estándar
    # de la población, así como el máximo y el mínimo. 
    # Estos datos se muestran por defecto en la salida en modo texto
    # (se puede apagar),pero se pueden usar para graficar (ver tutorial)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    # Algoritmo genético con parámetros a usar y estadísticas a calcular    
    algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 50, stats=stats, 
                        halloffame=hof)
    
    return pop, stats, hof

### Ejecutar una búsqueda con el AG instanciado

Hacer pruebas cambiando los parámetros, número de iteraciones, semilla, etc.

In [9]:
_, _, _ = evolve()

gen	nevals	avg    	std    	min 	max 
0  	300   	4690.49	445.191	3481	5841
1  	226   	4443.21	399.971	3129	5941
2  	226   	4346.49	411.386	3129	5692
3  	220   	4305.31	459.031	3129	5657
4  	224   	4191.19	442.272	3129	5461
5  	208   	4123.49	488.654	3129	5441
6  	220   	4063.79	480.899	3129	5413
7  	230   	4042.96	488.855	3139	5372
8  	254   	4069   	512.454	3005	5676
9  	212   	4007.65	475.294	3005	5337
10 	236   	4031.02	555.703	3005	5478
11 	219   	3967.61	532.231	3005	5607
12 	215   	3918.59	540.498	3005	5501
13 	217   	3958.57	588.745	2665	5628
14 	217   	3844.71	565.507	2640	5287
15 	241   	3904.27	577.263	2640	5462
16 	243   	3907.89	506.703	2640	5367
17 	234   	3869.57	550.245	2595	5360
18 	232   	3811.59	545.775	2595	5329
19 	227   	3758.08	595.753	2537	5350
20 	227   	3676.42	603.626	2465	5252
21 	214   	3642.24	624.912	2465	5113
22 	244   	3633.23	635.848	2465	5838
23 	223   	3515.93	566.066	2447	4986
24 	217   	3477.36	664.356	2447	5114
25 	244   	3416.73	591.752	2465	5112
2